## Late merge with hard parameters ##

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import  Dataset
from PIL import Image
from torchvision import transforms
import random
from sklearn.metrics import accuracy_score

In [2]:
class lmerge_dataloader(Dataset):
    def __init__(self, path , train):
        self.path = path
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.data_lister(train)
        self.length = len(self.dataset)
    def data_lister(self, train): 
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.path, class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        random.shuffle(data_list)
        if train:
            return data_list[:int(0.8*len(data_list))]
        else:
            return data_list[int(0.8*len(data_list)):]
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png')).convert('L')) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '1.png')).convert('L')) # image 2 in tensor
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
        ])
        pic1 = transform(pic1)
        pic2 = transform(pic2)
        
        return pic1 , pic2, sample_class

In [3]:
late_hard_training = lmerge_dataloader('./dataset/', True)
late_hard_testing = lmerge_dataloader('./dataset/', False)

In [4]:
print(len(os.listdir("./dataset/./NotSnow/")))
print(len(os.listdir("./dataset/./Snow/")))

2518
1049


In [5]:
sample = late_hard_training[1311]
print(sample[0].shape)
print(sample[1].shape)
print(sample[2])

torch.Size([1, 224, 224])
torch.Size([1, 224, 224])
0


In [6]:
class late_merge_NN(nn.Module):
    def __init__(self):
        super(late_merge_NN, self).__init__()
        self.convolve1 = nn.Conv2d(1, 2, 1)
        self.convolve2 = nn.Conv2d(2, 4, 1)
        self.pool = nn.MaxPool2d(kernel_size=4, stride=5)
        self.convolve12 = nn.Conv2d(1, 2, 1)
        self.convolve22 = nn.Conv2d(2, 4, 1)
        self.pool2 = nn.MaxPool2d(kernel_size=4, stride=5)
        self.full1 = nn.Linear(648, 2)

    def forward(self, x1, x2):
        x1 = self.convolve1(x1)
        x1 = self.pool(x1)
        x1 = self.convolve2(x1)
        x1 = self.pool(x1)
        x1 = x1.view(-1, 324)
        
        x2 = self.convolve12(x2)
        x2 = self.pool2(x2)
        x2 = self.convolve22(x2)
        x2 = self.pool2(x2)
        x2 = x2.view(-1, 324)

        x = torch.cat((x1, x2), dim=1)
        x = self.full1(x)
        return x


In [7]:
# hyper parameters
learning_rate_2= 0.001
epochs_2 = 5

In [8]:
# training the neural net with the stacking dataset
model2 = late_merge_NN()
criterion2 = nn.CrossEntropyLoss()
optimiser2 = optim.Adam(model2.parameters(), lr = learning_rate_2)

In [9]:
# model params
total_params = sum(p.numel() for p in model2.parameters())
print(f'the total number of parameters in this model are {total_params}')

the total number of parameters in this model are 1330


In [10]:
model2.train()
for epoch in range(epochs_2):
    running_loss = 0 
    predicted_list = []
    labels_list = []
    print(f'this is epoch number {epoch}')
    for i, (input1,input2, label) in enumerate(late_hard_training):
        labels_list.append(label)
        optimiser2.zero_grad()
        label = torch.tensor(label).view(-1)
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model2(input1, input2)
        loss = criterion2(outputs, label)
        loss.backward()
        optimiser2.step()
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.item())
        running_loss += loss.item()
        if i % 1000 == 999:   
            print('loss: %.3f accuracy: %.3f' %(running_loss / 10, 100 *accuracy_score(labels_list, predicted_list)))
            running_loss = 0.
           

this is epoch number 0
this is the 0 iteration
loss: 47.122 accuracy: 73.600
this is the 1000 iteration
loss: 30.090 accuracy: 79.750
this is the 2000 iteration
this is epoch number 1
this is the 0 iteration
loss: 28.197 accuracy: 87.900
this is the 1000 iteration
loss: 24.406 accuracy: 89.050
this is the 2000 iteration
this is epoch number 2
this is the 0 iteration
loss: 25.079 accuracy: 89.300
this is the 1000 iteration
loss: 21.728 accuracy: 90.400
this is the 2000 iteration
this is epoch number 3
this is the 0 iteration
loss: 22.926 accuracy: 90.900
this is the 1000 iteration
loss: 19.719 accuracy: 91.800
this is the 2000 iteration
this is epoch number 4
this is the 0 iteration
loss: 21.081 accuracy: 92.200
this is the 1000 iteration
loss: 18.008 accuracy: 92.900
this is the 2000 iteration


In [11]:
model2.eval()
loss = 0 
correct = 0
total = 0
count = 0
predicted_list = []
labels_list = []
with torch.no_grad():
    for i, (input1,input2, label) in enumerate(late_hard_testing):
        label = torch.tensor(label)
        label = torch.tensor(label).view(-1)
        labels_list.append(label.item())
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model2(input1, input2)
        loss += criterion2(outputs, label).item()
        predicted = outputs.argmax(dim = 1 , keepdim = True)
        predicted_list.append(predicted.item())


/var/folders/l8/nqt3x5yn28z6g3mdkkcd_w940000gn/T/ipykernel_32148/2927117641.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).view(-1)


this is the 0 iteration


In [12]:
print(f'the model accuracy is {accuracy_score(labels_list, predicted_list)}')

the model accuracy is 0.9411764705882353
